In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

In [2]:
probs_cols = [
        "censys",
        "driftnet",
        "internetcensus",
        "intrinsec",
        "ipip",
        "mirai",
        "onyphe",
        "rapid7",
        "securitytrails",
        "shadowserver",
        "shodan",
        "u_mich",
        "unk_bruteforcer",
        "unk_exploiter",
        "unk_spammer",
        "unknown"
    ]

probs_cols.sort()
idx_to_label = { idx: label for idx, label in enumerate(probs_cols) }
idx_to_label

{0: 'censys',
 1: 'driftnet',
 2: 'internetcensus',
 3: 'intrinsec',
 4: 'ipip',
 5: 'mirai',
 6: 'onyphe',
 7: 'rapid7',
 8: 'securitytrails',
 9: 'shadowserver',
 10: 'shodan',
 11: 'u_mich',
 12: 'unk_bruteforcer',
 13: 'unk_exploiter',
 14: 'unk_spammer',
 15: 'unknown'}

In [3]:
def convert(y, idx_to_label):
    
    return [ idx_to_label[idx] for idx in y ]

# Function to extract 'precision' values and calculate mean
def calculate_mean_f1(row):
    f1s = [cell['f1-score'] for cell in row]
    return sum(f1s) / len(f1s)

In [4]:
CASE = 3

In [5]:
with open(f"data/2022/output/nn/{CASE}/preds.pkl", "rb") as fd:
    preds = pickle.load(fd)

In [6]:
preds.keys(), preds["20221021"].keys(), preds["20221021"][0].keys()

(dict_keys(['20221021', '20221022', '20221023', '20221024', '20221025', '20221026', '20221027', '20221028', '20221029', '20221030', '20221031']),
 dict_keys([np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)]),
 dict_keys(['y_test', 'y_hat']))

In [7]:
days = list(preds.keys())
days.sort()

In [8]:
scores = {}
for day in days:
    dfs = []
    for fold in np.arange(10):
        y = convert(preds[day][fold]["y_test"], idx_to_label)
        y_hat = convert(preds[day][fold]["y_hat"], idx_to_label)
        report = classification_report(y, y_hat, labels=probs_cols, zero_division=0, output_dict=True)
        dfs.append(report)
    scores[day] = pd.DataFrame(dfs)[probs_cols + ["macro avg"]].T.apply(lambda x: calculate_mean_f1(x), axis=1)

In [9]:
scores["20221021"]

censys             0.934578
driftnet           0.958327
internetcensus     0.946953
intrinsec          0.300000
ipip               0.250000
mirai              0.989789
onyphe             0.912092
rapid7             0.000000
securitytrails     1.000000
shadowserver       0.991406
shodan             0.726667
u_mich             1.000000
unk_bruteforcer    0.971449
unk_exploiter      0.905455
unk_spammer        0.831693
unknown            0.000000
macro avg          0.732401
dtype: float64

In [10]:
pd.DataFrame(scores).T.mean(axis=0).apply(lambda x: np.trunc(100 * x) / 100)

censys             0.93
driftnet           0.95
internetcensus     0.96
intrinsec          0.11
ipip               0.18
mirai              0.98
onyphe             0.66
rapid7             0.36
securitytrails     0.81
shadowserver       0.99
shodan             0.68
u_mich             0.35
unk_bruteforcer    0.95
unk_exploiter      0.89
unk_spammer        0.82
unknown            0.00
macro avg          0.66
dtype: float64

In [11]:
set([1,2,3]).intersection(set([1,2,4]))

{1, 2}